# Roadmap
$$\text{Multineuron Timeseries: } (T * B) \times N$$
$$\downarrow \text{Z-Score (Neurons)}$$
$$\downarrow PCA$$ 
$$\text{PCA Scores: } (T * B) \times N \xrightarrow{\text{reshape}} \text{Relative Response: } T \times (N * B)$$
$$\downarrow{\frac{\Sigma{\text{ (columns: T)}}} {\text{T in E}}} $$
$$PSTH$$
<br>
$\text{T = trials | N = neurons | B = bins | E = events}$

# Goal
* Format the data into the multineuron timeseries format
* Perform PCA on the neural data and extract out the scores
  * Reshape scores back into PCA
  * Run classification on first 4 principal components

$$\pagebreak$$

# What is Principal Component Analysis (PCA)?
* PCA = dimension reduction technique
* Works by finding componenets that maximally explain variance within data
  * Number of input features = max of possible components that can come out

# PCA Visual Depiction
![PCA Example Plot](images/pca_example_cropped.png)

# PCA Behind the Scenes
1. Calculate covariance matrix  
    * diagonal = variance
    * off-diagonal = covariance
2. Use Singular Value Decomposition (SVD) on covariance matrix  
    * SVD is an eigendecomposition algorithm and gives the eigenvalues and eigenvectors of the matrix
    * Nature of algorithm organizes components in decreasing order of variance explained
3. Use eigenvalues and eigenvectors to transform data into PC space
    * Weighted sum of your features (neurons) on each data point in your pca input
4. Done!

$$\pagebreak$$

# Relative Response Matrix
* Rows = trials
* Columns = Time grouped by Neuron
$$\text{event 1 relative response for 2 Neurons}$$
$$
\begin{bmatrix}\text{trial 1}\\\text{trial 2}\end{bmatrix}
\begin{bmatrix}0&2&4&3\\0&1&0&1\end{bmatrix}
\begin{bmatrix}0&2&1&3\\0&0&2&1\end{bmatrix}
$$
<br>
$$\text{event 2 relative response for 2 Neurons}$$
$$
\begin{bmatrix}\text{trial 1}\end{bmatrix}
\begin{bmatrix}0&0&2&3\end{bmatrix}
\begin{bmatrix}1&0&2&3\end{bmatrix}
$$

# PCA Input (multineural timeseries, MNTS)
* Rows = Time grouped by trial (observations)
* Columns = Neurons (features)
$$
\begin{bmatrix}
event 1\\event 1\\event 1\\event 1\\event 1\\event 1\\event 1\\event 1\\
event 2\\event 2\\event 2\\event 2
\end{bmatrix}
\begin{bmatrix}
T1B1\\T1B2\\T1B3\\T1B4\\T2B1\\T2B2\\T2B3\\T2B4\\
T1B1\\T1B2\\T1B3\\T1B4
\end{bmatrix}
\begin{bmatrix}
0&0\\2&2\\4&1\\3&3\\0&0\\1&0\\0&2\\1&1\\
0&1\\0&0\\2&2\\3&3
\end{bmatrix}
$$

# MNTS Side Notes
* Make sure your PCA input contains all neurons and events in one matrix
  * The hope is that the events can be separated by the different componenets since they will have non overlapping variance in observations
  * Neurons are your feature inputs for PCA
* Keep track of event and trial order
  * After PCA you will get out a score matrix with data mapped to PC space
  * This transformed matrix will need to be reshaped into the relative response matrix
  * After reshaping you will then proceed with PSTH classification
* There are a few ways of creating the PCA input
  1. Directly creating the matrix while binning the data
      * New function like relative response function but stores data differently
  2. Reshaping the relative response matrix into the multineuron timeseries
      * The only difference between the two formats are how the bins are grouped

# Running PCA  
## Matlab
```m
% pca_input = input matrix
% rows = trials * bins
% columns = neurons
norm_pca_input = zscore(pca_input);
[loadings, pca_score, eigenvalues, ~, pc_variance, ~] = pca(norm_pca_input);
```  
## Python
```py
from scipy import stats
from sklearn.decomposition import PCA

# pca_input = input matrix
# rows = trials * bins
# columns = neurons
# Make sure that you remove any columns with all 0s before z-scoring
norm_pca_input = stats.zscore(pca_input, axis = 1, ddof = 1)
pca = PCA()
pca.fit(norm_pca_input)
pca_score = pca.transform(norm_pca_input)

eigenvalues = pca.explained_variance_.tolist()
pc_variance = (pca.explained_variance_ratio_ * 100).tolist()
```

# What to turn in
## Conceptual Questions (10pts)
* pdf with answers to conceptual questions
* Code files you wrote to create JSON output file
* Code files should be plain text with proper file extension
  * ex: last_first_hw4.m, last_first_hw4.py, etc.

# Json Output File (10pts)
* This file will be input into automatic grader
* This was taken from the example solution json and is a small snippet
* You are not required to turn in a pretty formatted json file
```json
{
    "psth_classifier": {
        "mutual_info": 0.32068603931534856,
        "performance": 0.46
    }
}
```